# [Handling Missing Values](https://www.kaggle.com/dansbecker/handling-missing-values)

There are many ways data can end up with missing values.    
* A 2 bedroom house doesn't have value for a third bedroom.
* Someone being surveyed may choose not to share their income.

Python libraries represent missing numbers as `nan` which is short for "not a number".  
You can detect which cells have missing values, and then count how many there are in each column with the command:

Most libraries (including scikit-learn) will give you an error if you try to build a model using data with missing values.  
Let's figure out how to deal with them.